# Evaluation of Reconstruction Performance

In [1]:
import sys
import os
from os.path import join
parent_dir = os.path.abspath(join(os.getcwd(), os.pardir))
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

from pathlib import Path
import torch as pt
from torch.utils.data import Subset
from torch.nn.functional import mse_loss
from CNN_VAE import ConvEncoder, ConvDecoder, Autoencoder
from utils.training_loop import train_cnn_vae
import utils.config as config
import matplotlib.pyplot as plt

pt.manual_seed(0)

plt.rcParams["figure.dpi"] = 180

# use GPU if possible
device = pt.device("cuda:0") if pt.cuda.is_available() else pt.device("cpu")
print(device)

DATA_PATH = Path(os.path.abspath('')).parent / "data"
OUTPUT_PATH = Path(os.path.abspath('')).parent / "output" / "VAE"
MODEL_PATH = Path(os.path.abspath('')).parent / "output" / "VAE" / "latent_study"

cpu


In [2]:
# load test dataset
test_dataset = pt.load(join(DATA_PATH, "test_dataset.pt"))

# split test dataset into the two flow conditions
X_test_1 = Subset(test_dataset,                                 # ma0.84 alpha3.00 
                  list(range(0, int(len(test_dataset) / 2))))        
X_test_2 = Subset(test_dataset,                                 # ma0.84 alpha5.00
                  list(range(int(len(test_dataset) / 2), len(test_dataset))))    

# make tensors from datasets
X_test_1_tensor = pt.stack([X_test_1[i] for i in range(len(X_test_1))], dim=3).squeeze(0)
X_test_2_tensor = pt.stack([X_test_2[i] for i in range(len(X_test_2))], dim=3).squeeze(0)
print(X_test_1_tensor.shape)

torch.Size([256, 128, 1])


#### MSE and Variance Reconstruction with varying number of bottleneck neurons

In [5]:
# Initialize lists to save the computed metrics
MSE_1 = []
MSE_2 = []
Var1 = []
Var2 = []

# compute the total variance of test datasets
orig_Var1 = pt.var(X_test_1_tensor)
orig_Var2 = pt.var(X_test_2_tensor)

# scan directory for trained models and extract paths as well as the latent size of the model
dirs = [os.path.join(MODEL_PATH, name, name) for name in os.listdir(MODEL_PATH) if os.path.isdir(os.path.join(MODEL_PATH, name))]
sorted_dirs = sorted(dirs, key=lambda x: int(os.path.basename(x)))
latent_sizes = [int(os.path.basename(dir)) for dir in sorted_dirs]

In [ ]:
# function to create VAE model
def make_VAE_model(n_latent: int = 256) -> pt.nn.Module:
    encoder = ConvEncoder(
        in_size=config.target_resolution,
        n_channels=config.input_channels,
        n_latent=config.latent_size,
        variational=True,
        layernorm=True
    )

    decoder = ConvDecoder(
        in_size=config.target_resolution,
        n_channels=config.output_channels,
        n_latent=config.latent_size,
        layernorm=True,
        squash_output=True
    )

    autoencoder = Autoencoder(encoder, decoder)
    autoencoder.to(device)
    return autoencoder

In [6]:
for i, latent_size in enumerate(latent_sizes):
    print(sorted_dirs[i])
    # load model
    autoencoder = make_VAE_model(latent_size)
    autoencoder.load(sorted_dirs[i])

    # reconstruct test dataset 1 and compute MSE
    reconstructed = pt.stack([autoencoder(X_test_1[i]) for i in range(len(X_test_1))], dim=3).squeeze(0)
    MSE_1.append(mse_loss(X_test_1_tensor, reconstructed).item())

    # compute variance reconstruction
    Var1.append((1 - ((orig_Var1 - pt.var(reconstructed)) / orig_Var1)) * 100)

/home/anton/repositories/Studienarbeit/output/VAE/latent_study/0/0


RuntimeError: Error(s) in loading state_dict for Sequential:
	Unexpected key(s) in state_dict: "_layers.0.weight", "_layers.0.bias", "_layers.1.weight", "_layers.1.bias", "_layers.2.weight", "_layers.2.bias", "_layers.3.weight", "_layers.3.bias", "_layers.4.weight", "_layers.4.bias", "_layers.5.weight", "_layers.5.bias", "_layers.6.weight", "_layers.6.bias", "_layers.7.weight", "_layers.7.bias", "_latent_mean.weight", "_latent_mean.bias", "_latent_log_var.weight", "_latent_log_var.bias". 

In [ ]:
# Plot the results and save the figure
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(latent_sizes, MSE_1, label="Test Dataset 1")
ax1.plot(latent_sizes, MSE_2, label="Test Dataset 2")
ax1.set_title("MSE")
ax2.plot(latent_sizes, Var1, label="Test Dataset 1")
ax2.plot(latent_sizes, Var2, label="Test Dataset 2")
ax2.set_title("Variance Reconstruction in %")
ax2.set_xlabel("number of bottleneck neurons")
ax2.set_xticks(range(0, 325, 25))
handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles, labels)
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, "MSE_and_Variance_with_latent_size.png"), bbox_inches = "tight")

#### Temporal MSE distribution with varying number of bottleneck neurons

#### Spatial MSE distribution with varying number of bottleneck neurons

#### Reconstructed pressure field compared to Ground Truth for two bottleneck sizes